In [1]:
import os
from glob import glob as g
import pandas as pd
import numpy as np
import shutil
import pyxlsb
current_directory = os.getcwd()

In [2]:
country = 'CR'
production_quarter = 'FY24Q3'
previous_qaurter = 'FY24Q2'

In [3]:
# update new import file for this quarter
from utils import club_raw_files

new_raw_files = os.path.join(current_directory, f'Latest raw import files\\{country}')
new_unmapped_data = club_raw_files(new_raw_files)
new_unmapped_data.head(3)

,Date,Day,Month,Year,Declaration_Number,Importer_Code,Importer,Origin_Country,Shipping_Country,Sales_Country,...,Declarant_Code,Declarant,Total_Invoice_Value,Sum_of_Total_Value,Total_Gross_Weight_Kg,Total_Net_Weight_Kg,Exchange_Rate,Remarks,Bulks,Mode_of_Transport_EN
0,2023-11-01,1,11,2023,2023-001-079703,310279705402,POCR LLC SOCIEDAD DE RESPONSABILIDAD LIMITADA,COSTA RICA,COSTA RICA,COSTA RICA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-01,1,11,2023,2023-005-650849,310235746925,BOSTON SCIENTIFIC DE COSTA RICA SOCIEDAD,COSTA RICA,COSTA RICA,COSTA RICA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-11-01,1,11,2023,2023-005-650846,310112350729,DUWEST COSTA RICA SA,UNITED STATES,UNITED STATES,UNITED STATES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
new_unmapped_data[['Month','Year']].drop_duplicates()

,Month,Year
0,11,2023
0,July,2023
35459,october,2023
72478,august,2023
111017,september,2023


In [5]:
month_alignment = {11:'202311','July':'202307','october':'202310','august':'202308','september':'202309'}
new_unmapped_data['Month'] = new_unmapped_data['Month'].map(month_alignment)
new_unmapped_data[['Month','Year']].drop_duplicates()

,Month,Year
0,202311,2023
0,202307,2023
35459,202310,2023
72478,202308,2023
111017,202309,2023


In [6]:
cols_to_rename =  {
                   'Product_Description':'Detailed_Product',
                   'Product_Description_EN':'Detailed_Product_EN','Total_Value_USD':'Total Value_USD',
                   'Quantity':'Sta_Quantity', 'Unit':'Unit_Sta_Qty-Raw','Unit_EN':'Unit_Sta_Qty-Standardised', 'Supplier':'Exporter'
       }
new_unmapped_data.rename(columns=cols_to_rename, inplace=True)
new_unmapped_data.columns

Index(['Date', 'Day', 'Month', 'Year', 'Declaration_Number', 'Importer_Code',
       'Importer', 'Origin_Country', 'Shipping_Country', 'Sales_Country',
       'Port_of_Discharge_Code', 'Port_of_Discharge', 'HS_Code',
       'Detailed_Product', 'Detailed_Product_EN', 'Packages',
       'Total Value_USD', 'Gross_Weight_Kg', 'Net_Weight_Kg', 'Serial_Number',
       'Shipping_Number', 'Import_Duty_CRD', 'QUANTITY', 'UNIT', 'Chapter',
       'Heading', 'Sub_Heading', 'Package_Unit', 'Brand', 'Mode_of_Transport',
       'Type', 'Importer_Address', 'Declarant_Code', 'Declarant',
       'Total_Invoice_Value', 'Sum_of_Total_Value', 'Total_Gross_Weight_Kg',
       'Total_Net_Weight_Kg', 'Exchange_Rate', 'Remarks', 'Bulks',
       'Mode_of_Transport_EN'],
      dtype='object')

In [7]:
historical_import_data_file = g(os.path.join(current_directory, 
                                            f'Import data warehouse\\Historical Import Data\\*{country}*.csv'))[0]
historical_import_data = pd.read_csv(historical_import_data_file)
historical_import_data[['MDT FS Quarter_Import Lag', 'Calendar Quarter', 'Month', 'Year']].drop_duplicates()

C:\Users\AbhishekSrivastava\AppData\Local\Temp\ipykernel_10484\1340150618.py:3: DtypeWarning: Columns (17,18,19,20,21,22,28,29,30,31,33,34,37,40) have mixed types. Specify dtype option on import or set low_memory=False.
  historical_import_data = pd.read_csv(historical_import_data_file)


,MDT FS Quarter_Import Lag,Calendar Quarter,Month,Year
0,2022Q3,2022Q1,202201,2022
8,2022Q4,2022Q1,202202,2022
10,2022Q4,2022Q1,202203,2022
12,2023Q1,2022Q3,202207,2022
13,2023Q2,2022Q3,202208,2022
78,2022Q4,2022Q2,202204,2022
81,2023Q1,2022Q2,202206,2022
125,2023Q2,2022Q3,202209,2022
146,2023Q2,2022Q4,202210,2022
159,2023Q3,2022Q4,202211,2022


In [8]:
Import_data = pd.concat([historical_import_data, new_unmapped_data])
Import_data.shape

(776706, 77)

In [9]:
Import_data.columns

Index(['Index', 'HS_Code', 'Detailed_Product', 'Detailed_Product_EN',
       'Importer', 'Importer_Standardised', 'Exporter',
       'Exporter_Standardised', 'Unit_Sta_Qty-Raw',
       'Unit_Sta_Qty-Standardised', 'Sta_Quantity', '# Unit per Sta_Quantity',
       'Total Volume-Standardised_Unit', 'Total Value_USD', 'Total Value_LCU',
       'Unit Price_USD-Sta_Quantity', 'Unit Price_USD-Standardised_Unit',
       'Operating Unit', 'Sub-Operating Unit', 'Product Type', 'Company',
       'Family name', 'Keyword Suggested', 'Keyword QC',
       'MDT FS Quarter_Import Lag', 'Calendar Quarter', 'Month', 'Year',
       'Date', 'Python_Mapped_Device', 'Python_Mapped_Family_Name',
       'Python_Mapped_Manufacturer_Name', 'PACKAGES', 'DECLARATION_NUMBER',
       'ORIGIN_COUNTRY', 'CHAPTER', 'Day', 'IMPORTER_CODE', 'NET_WEIGHT_KG',
       'GROSS_WEIGHT_KG', 'MODE_OF_TRANSPORT', 'Heading', 'Sub_Heading',
       'MDT FS Quarter', 'MDT Fiscal Year', 'HS_sub', 'Declaration_Number',
       'Importer

In [10]:
Import_data['HS_Code'] = Import_data['HS_Code'].astype(str)
Import_data['HS_sub'] = Import_data['HS_Code'].str[:4]
Import_data['Detailed_Product_EN'] = Import_data['Detailed_Product_EN'].astype(str).str.lower()
Import_data['Index']=range(1,len(Import_data)+1)

In [11]:
keyword_file = g(os.path.join(current_directory, 'Keyword List\\*.*'))[0]
keyword_file

'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Surgicals\\Mexsurg Latamsurg\\Import Data Processing\\Import data processing pipeline\\Keyword List\\Import_Keyword_10 Aug 2023.xlsx'

In [15]:
import warnings as ws
ws.filterwarnings("ignore")
raw_data_drop = map_using_keyword_list(Import_data, keyword_file)

Number of rows to process: 156314
Number of keywords to match: 10599
First Round of mapping...


100%|████████████████████████████████████████████████████████████████████████████| 10599/10599 [42:51<00:00,  4.12it/s]


Second Round of mapping...


100%|████████████████████████████████████████████████████████████████████████████████| 522/522 [02:32<00:00,  3.42it/s]


Third Round of mapping...


100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [00:11<00:00,  7.33it/s]


NameError: name 'raw_data_drop' is not defined

In [13]:


def map_using_keyword_list(Import_data, keyword_file):
    device_list_first = pd.read_excel(keyword_file, sheet_name="First Script", usecols="A:F")
    device_list_second= pd.read_excel(keyword_file, sheet_name="Second Script", usecols="A:E")
    device_list_third= pd.read_excel(keyword_file, sheet_name="Third Script", usecols="A:C")

    device_list_first['Product_Name_short']=device_list_first.Product_Name_short.astype(str)
    device_list_first['Product_Name_short']=device_list_first.Product_Name_short.str.lower()
    device_list_first['Product_Name_short']=device_list_first.Product_Name_short.str.replace('-', ' ')

    device_list_second['Keyword_Mapped']=device_list_second.Keyword_Mapped.str.replace('-', ' ')
    device_list_second['Keyword_Mapped']=device_list_second.Keyword_Mapped.astype(str)
    device_list_second['Keyword_Mapped']=device_list_second.Keyword_Mapped.str.lower()

    device_list_second['second_Keyword']=device_list_second.second_Keyword.astype(str)
    device_list_second['second_Keyword']=device_list_second.second_Keyword.str.lower()
    device_list_second['second_Keyword']=device_list_second.second_Keyword.str.replace('-', ' ')

    device_list_third['HS_sub']=device_list_third.HS_sub.astype(str)
    device_list_third['Product_Type']=device_list_third["Product_Type"].astype(str)
    
    
    unique_import_data_descriptions =  Import_data.drop_duplicates(subset=['Detailed_Product_EN'])
    print(f"Number of rows to process: {len(unique_import_data_descriptions)}")
    
    device_list_first = device_list_first.replace(np.nan, '', regex=True)
    keywords=device_list_first.to_dict('records')
    print(f"Number of keywords to match: {len(keywords)}")
    
    ## Create columns for mappings
    columns_to_add = ['Python_Mapped_Device', 'Python_Mapped_First_Keyword', 'Python_Mapped_Family_Name', 'Python_Mapped_Manufacturer_Name']
    for col in columns_to_add:
        unique_import_data_descriptions[col] = None
        
   # Assuming raw_data, keywords_data, device_list_second, device_list_third, and raw_data_drop are already defined

    # First round of mapping
    print(f"First Round of mapping...")
    mapped_data1 = map_first_round(unique_import_data_descriptions.copy(), device_list_first.to_dict('records'))

    # Second round of mapping
    print(f"Second Round of mapping...")
    mapped_data2 = map_second_round(mapped_data1.copy(), device_list_second.to_dict('records'))

    # Third round of mapping
    print(f"Third Round of mapping...")
    mapped_data3 = map_third_round(mapped_data2.copy(), device_list_third.to_dict('records'))

    # Splitting 'Python_Mapped_Device' into 'Segment'
    final_mapped_data = split_device_segment(mapped_data3)

    # Final concatenation and deduplication
    final_result = final_concatenation(raw_data_drop, final_mapped_data)


    return final_result
    
    

In [14]:
import pandas as pd
from tqdm import tqdm
def map_first_round(raw_data, keywords_data):
    # Use a boolean series to track processed rows
    processed = pd.Series(False, index=raw_data.index)
    for keyword in tqdm(keywords_data):
        #print(keyword)
        mask = raw_data['Detailed_Product_EN'].fillna("missing").str.contains(keyword['Product_Name_short'], case=False)
        # Avoid reprocessing already processed rows
        mask &= ~processed
        raw_data.loc[mask, 'Python_Mapped_Device'] = keyword['Product_Type']
        raw_data.loc[mask, 'Python_Mapped_First_Keyword'] = keyword['Product_Name_short']
        raw_data.loc[mask, 'Python_Mapped_Family_Name'] = keyword['Product_Name']
        raw_data.loc[mask, 'Python_Mapped_Manufacturer_Name'] = keyword['Company_Name_short']
        # Mark these rows as processed
        processed |= mask
    return raw_data.drop_duplicates(subset="Index", keep="first")

def map_second_round(mapped_data, device_list_second):
    for row in tqdm(device_list_second):
        mask = (mapped_data['Python_Mapped_First_Keyword'].fillna("missing").str.contains(row['Keyword_Mapped'], case=False)
                & mapped_data['Detailed_Product_EN'].str.contains(row['second_Keyword'], case=False))
        mapped_data.loc[mask, 'Python_Mapped_Device'] = row['Product Type Mapped']
        mapped_data.loc[mask, 'Python_Mapped_Second_Keyword'] = row['second_Keyword']
        mapped_data.loc[mask, 'Python_Mapped_Family_Name'] = row['Product_Name']
        mapped_data.loc[mask, 'Python_Mapped_Manufacturer_Name'] = row['Company_Name_short']
    return mapped_data.drop_duplicates(subset="Index", keep="first")

def map_third_round(result, device_list_third):
    for row in tqdm(device_list_third):
        mask = (result['HS_sub'].fillna("missing").str.contains(row['HS_sub'], case=False)
                & result['Python_Mapped_Device'].str.contains(row['Product_Type'], case=False))
        result.loc[mask, 'Python_Mapped_Device'] = row['Product_Type_Update']
    return result.drop_duplicates(subset="Index", keep="first")

def split_device_segment(result):
    new = result["Python_Mapped_Device"].str.split("_", n=1, expand=True)
    result["Segment"] = new[0]
    return result

def final_concatenation(raw_data_drop, final_result):
    result3 = pd.concat([raw_data_drop, final_result]).drop_duplicates(subset="Index", keep="last")
    if result3.shape[0] != raw_data_drop.shape[0]:
        raise ValueError("Error, row counts do not match. Please recheck the code.")
    return result3


In [10]:

!pip install pyahocorasick


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
